In [ ]:
import json
from itertools import combinations
from typing import List, Dict, Any


def load_program_data(filepath: str) -> List[Dict[str, Any]]:
    print(f"\n📥 Loading programs from '{filepath}'...")
    with open(filepath, 'r', encoding='utf-8') as f:
        programs = json.load(f)
    print(f"✅ Loaded {len(programs)} programs.\n")
    return programs

def generate_valid_combinations(program: Dict[str, Any], student_electives_input: List[str]) -> List[List[str]]:
    print(f"\n🔍 Checking program: {program['program']}")
    
    if program["campus"] == "Obuasi Campus":
        print("❌ Skipping program at Obuasi Campus.")
        return []

    else:
        # Get the program's elective requirements
        elective_reqs = program.get("elective subjects", {})
        main = elective_reqs.get("main")
        tracks = elective_reqs.get("tracks", {})

        # Collect all the required subjects for the program
        all_required_subjects = set()

        print("🧑‍🏫 Processing elective requirements:")
        
        if main:
            if isinstance(main, list):
                all_required_subjects.update(main)
            else:
                all_required_subjects.add(main)
        print(f"   ➡ Main subject(s) added: {main}")

        for track_name, track in tracks.items():
            for group in track:
                if isinstance(group, list):
                    all_required_subjects.update(group)
                    print(f"   ➡ Track '{track_name}' - Adding mutually exclusive group: {group}")
                else:
                    all_required_subjects.add(group)
                    print(f"   ➡ Track '{track_name}' - Adding subject: {group}")
        
        print(f"📘 All required subjects: {sorted(all_required_subjects)}")
        print(f"🎓 Student electives input: {student_electives_input}")

        # Find matching electives between student input and program requirements
        matching_electives = [s for s in student_electives_input if s in all_required_subjects]
        print(f"🔎 Matching electives: {matching_electives}")

        if len(matching_electives) < 3:
            print("❌ Not enough matching electives to form 3-subject combinations.")
            return []

        # Check if the main subject is required and included
        if main:
            main_set = set(m.strip().lower() for m in main) if isinstance(main, list) else {main.strip().lower()} if main else set()
            print(f"🧪 Main subject(s) set for comparison: {main_set}")
        else:
            main_set = set()

        valid_combinations_set = set()

        # Generate all 3-subject combinations and check each one
        for combo in combinations(matching_electives, 3):
            combo_set = frozenset(combo)
            print(f"🔄 Checking combination: {combo_set}")

            # Check for mutually exclusive electives in the same combination
            mutually_exclusive_found = False
            for track_name, track in tracks.items():
                for group in track:
                    if isinstance(group, list):
                        # Check if two or more subjects from the same group are found in the combo
                        common_subjects = [item for item in group if item in combo_set]
                        if len(common_subjects) > 1:  # More than 1 subject from the same mutually exclusive group
                            mutually_exclusive_found = True
                            print(f"⛔ Mutually exclusive combination found in track '{track_name}': {combo} (Subjects: {common_subjects})")
                            break
                if mutually_exclusive_found:
                    break

            if mutually_exclusive_found:
                continue

            # Check if the combination includes the main subject (if required)
            if main and not any(sub.strip().lower() in main_set for sub in combo):
                print(f"⛔ Skipping combo (missing main subject): {combo}")
                continue
            
            valid_combinations_set.add(combo_set)
            print(f"✅ Valid combination: {combo}")

        # Convert to sorted list of lists
        valid_combinations = [sorted(list(combo)) for combo in valid_combinations_set]

        print(f"🎯 Total valid combinations: {len(valid_combinations)}")
        return valid_combinations


In [11]:
# === Run the checker ===
if __name__ == '__main__':
    student_electives = ["Chemistry", "Physics", "Elective Mathematics", "Biology", "General Agriculture", "Economics"]
    program_list = load_program_data('data/programdatan.json')

    for program in program_list:
        combos = generate_valid_combinations(program, student_electives)
        print(combos)
        if combos:
            print(f"\n✅ {program['program']} - QUALIFIED")
            for combo in combos:
                print("   →", combo)
        else:
            print(f"\n❌ {program['program']} - NOT QUALIFIED")
            print("   → No valid combinations found.")
    print("\n📊 All programs processed.\n")


📥 Loading programs from 'data/programdatan.json'...
✅ Loaded 2 programs.


🔍 Checking program: BSc. Agribusiness Management
🧑‍🏫 Processing elective requirements:
   ➡ Main subject(s) added: None
   ➡ Track 'General Science' - Adding subject: Chemistry
   ➡ Track 'General Science' - Adding mutually exclusive group: ['Physics', 'Elective Mathematics']
   ➡ Track 'General Science' - Adding mutually exclusive group: ['Biology', 'General Agriculture']
   ➡ Track 'Business' - Adding subject: Economics
   ➡ Track 'Business' - Adding subject: Accounting
   ➡ Track 'Business' - Adding subject: Business Management
   ➡ Track 'Business' - Adding subject: Business Mathematics
   ➡ Track 'Business' - Adding subject: Costing
   ➡ Track 'Business' - Adding subject: Elective Mathematics
   ➡ Track 'General Arts' - Adding subject: Economics
   ➡ Track 'General Arts' - Adding subject: Geography
   ➡ Track 'General Arts' - Adding subject: Elective Mathematics
   ➡ Track 'General Agriculture' - Adding su

In [1]:
import json
import pandas as pd

In [8]:
with open('../src/data/obuasi.json', 'r', encoding='utf-8') as f:
    obuasi = json.load(f)
with open('../src/data/programdata.json', 'r', encoding='utf-8') as k:
    kumasi = json.load(k)

In [11]:
# Populate obuasi obuasi college, core subjects and elective subjects and applicant type fields with kumasi data if there have same program name
for program in obuasi:
    for kumasi_program in kumasi:
        if program['program'] == kumasi_program['program']:
            program['college'] = kumasi_program['college']
            program['core subjects'] = kumasi_program['core subjects']
            program['elective subjects'] = kumasi_program['elective subjects']
            program['applicant type'] = kumasi_program['applicant type']
            program["special requirements / general information"] = kumasi_program["special requirements / general information"]
            break

In [12]:
# Export the updated obuasi data to a new JSON file
with open('../src/data/obuasi_updated.json', 'w', encoding='utf-8') as f:
    json.dump(obuasi, f, indent=4)